In [1]:
import numpy as np
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import sqlalchemy import create_engine
import psycopg2
import os
POSTGRES_PASSWORD= os.getenv('POSTGRES_PASSWORD')

In [ ]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host ='postgres',
    port = 5432,
    db = 'congress'
))

In [ ]:
myquery = '''
SELECT directordername
FROM members
ORDER BY lastname
'''

members= pd.read_sql_query(myquery, con=engine)
memberlist=zip(members['directordername'], members['bioguideid'])

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [ ]:
mymarkdown = '''
congressional elections are ususally all about the big, national issues. But these representatives and senators work for local districts and states. Not all issues are the same everywhere. The purpose of this dashboard is to collect the public data from these sources:

* [Official API for the U.S Congress](https://api.congress.gov/)

* [Voteview](https://voteview.com/data)

* [Open Secrets]()

* [ProPublica's Data on Bills](www)

'''

In [12]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

 

app.layout = html.Div(
    [
        html.H1("Know your representatives in Congress") ,
        html.H2("Data collected from APIs from congress.gov, Open secrets, Voteview.com, and ProPublica"),
        html.H3("DS 6600: Data Engineering 1, UVA Data Science"),

        dcc.markdown(mymarkdown)

        dcc.Dropdown(options = members,
                    value = ''Alma S. Adams)
    ]
)

 

if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", port=8050, debug=True)